## VirtualBox telepítés:

In [16]:
%%file ./virtualbox-install.sh
#!/bin/sh
sudo apt-get update && sudo apt-get dist-upgrade && sudo apt-get autoremove
sudo apt-get -y install gcc make linux-headers-$(uname -r) dkms
wget -q https://www.virtualbox.org/download/oracle_vbox_2016.asc -O- | sudo apt-key add -
wget -q https://www.virtualbox.org/download/oracle_vbox.asc -O- | sudo apt-key add -
sudo sh -c 'echo "deb http://download.virtualbox.org/virtualbox/debian $(lsb_release -sc) contrib" >> /etc/apt/sources.list'
sudo apt-get update
sudo apt-get install virtualbox-5.2
VBoxManage -v
 https://download.virtualbox.org/virtualbox/5.2.8/

curl -O http://download.virtualbox.org/virtualbox/5.2.8/Oracle_VM_VirtualBox_Extension_Pack-5.2.8.vbox-extpack
sudo VBoxManage extpack install Oracle_VM_VirtualBox_Extension_Pack-5.2.8.vbox-extpack
VBoxManage list extpacks


Overwriting ./virtualbox-install.sh


In [ ]:
%%bash
chmod +x ./virtualbox-install.sh
xterm -e "./virtualbox-install.sh"

***

## Felügyelet nélküli ubuntu 16.04 telepítő (iso) készítése:

In [ ]:
%%sh
cd ~
#rm -f -R ~/ubuntu-iso
pwd
git clone https://github.com/englert/ubuntu-iso.git
cd ~/ubuntu-iso
gnome-terminal -e "sudo ./create-unattended-iso.sh"


***

## Ubuntu szerver telepítése Virtualboxon:

In [ ]:
%%sh
VM="ubu"
RAM="2048"
VRAM="16"
HDD="8192"
LAN="enp3s0"
ISO="ubuntuserver.iso"
VMs="VirtualBox VMs"
cd ~
VBoxManage createvm --name $VM --ostype "Linux_64" --register
VBoxManage modifyvm $VM --nic1 bridged --bridgeadapter1 $LAN
VBoxManage createhd --filename "$VMs/$VM/$VM.vdi" --size $HDD
VBoxManage storagectl $VM --name "SATA Controller" --add sata
VBoxManage storageattach $VM --storagectl "SATA Controller" --port 0 --device 0 --type hdd --medium "$VMs/$VM/$VM.vdi"
VBoxManage storagectl $VM --name "IDE Controller" --add ide
VBoxManage storageattach $VM --storagectl "IDE Controller" --port 0 --device 0 --type dvddrive --medium $ISO
VBoxManage modifyvm $VM --ioapic on
VBoxManage modifyvm $VM --boot1 dvd --boot2 disk --boot3 none --boot4 none
VBoxManage modifyvm $VM --memory $RAM --vram $VRAM

In [ ]:
%%sh
VM="ubu"
VBoxManage startvm $VM --type headless

***

## Ansible install:

In [17]:
%%file ./ansible-install.sh
#!/bin/bash
sudo apt-get update
sudo apt-get -y install software-properties-common
sudo apt-add-repository -y ppa:ansible/ansible
sudo apt-get update
sudo apt-get -y install ansible

Writing ./ansible-install.sh


In [18]:
%%sh
chmod +x ./ansible-install.sh
xterm -e "./ansible-install.sh"

In [19]:
!ansible --version

ansible 2.4.3.0
  config file = /etc/ansible/ansible.cfg
  configured module search path = ['/home/sis/.ansible/plugins/modules', '/usr/share/ansible/plugins/modules']
  ansible python module location = /usr/local/lib/python3.5/dist-packages/ansible
  executable location = /usr/local/bin/ansible
  python version = 3.5.2 (default, Nov 23 2017, 16:37:01) [GCC 5.4.0 20160609]


***

In [20]:
%%file ./hosts
localhost              ansible_connection=local

v112 ansible_host=192.168.0.112

u100 ansible_host=192.168.0.100 
u101 ansible_host=192.168.0.101 
u102 ansible_host=192.168.0.102 
u103 ansible_host=192.168.0.103 
u104 ansible_host=192.168.0.104 
u105 ansible_host=192.168.0.105 
u106 ansible_host=192.168.0.106 
u107 ansible_host=192.168.0.107 
u108 ansible_host=192.168.0.108 
u109 ansible_host=192.168.0.109 
u110 ansible_host=192.168.0.110 
u111 ansible_host=192.168.0.111 
u112 ansible_host=192.168.0.112 
u113 ansible_host=192.168.0.113
u114 ansible_host=192.168.0.114 
u115 ansible_host=192.168.0.115 
u116 ansible_host=192.168.0.116 
u117 ansible_host=192.168.0.117 
u118 ansible_host=192.168.0.118 
u119 ansible_host=192.168.0.119 
u120 ansible_host=192.168.0.120 
u121 ansible_host=192.168.0.121 
u122 ansible_host=192.168.0.122 
u123 ansible_host=192.168.0.123 
u124 ansible_host=192.168.0.124 
u125 ansible_host=192.168.0.125 
u126 ansible_host=192.168.0.126 
u127 ansible_host=192.168.0.127 
u128 ansible_host=192.168.0.128 
u129 ansible_host=192.168.0.129 
u130 ansible_host=192.168.0.130 

amazon1 ansible_host=52.59.217.167

docker1 ansible_host=www.unibox.hu

[vbox]
v112
[vbox:vars]
ansible_port=22 
ansible_connection=ssh 
ansible_python_interpreter=/usr/bin/python3
ansible_user=ubuntu


[tim]
u[101:130]
[tim:vars]
ansible_port=22 
ansible_connection=ssh 
ansible_python_interpreter=/usr/bin/python3
ansible_user=sis


[ubuntu]
server1

[ubuntu:vars]
ansible_port=22
ansible_connection=ssh
ansible_python_interpreter=/usr/bin/python3

[amazon]
amazon1

[amazon:vars]
ansible_port=22
ansible_connection=ssh
ansible_user=ubuntu
ansible_ssh_private_key_file=/home/sis/.ssh/amazon1.pem
ansible_python_interpreter=/usr/bin/python3



Overwriting ./hosts


In [21]:
%%file ./ansible.cfg
[defaults]
host_key_checking = false

Writing ./ansible.cfg


In [22]:
%%file ./upload_ssh-keys.yml
---
- hosts: v112
  remote_user: ubuntu
  tasks:
  - name: upload ssh key                
    authorized_key:
      user: ubuntu
      state: present
      manage_dir: yes
      key: https://github.com/englert.keys


Overwriting ./upload_ssh-keys.yml


In [ ]:
%%bash
xterm -e "ansible-playbook -i ./hosts ./upload_ssh-keys.yml -k"

In [23]:
%%file ./sudo-group.j2
# {{ ansible_managed }}
%sudo ALL=(ALL) NOPASSWD:ALL

Writing ./sudo-group.j2


In [24]:
%%file ./passwordless-sudo.yml
---
- hosts: v112
  #remote_user: ubuntu
  tasks:

  - name:  Legyen sudo csoport
    group:
      name: sudo
      state: present

  - name: A sudo csoport felhasználói jelszó nélkül használhatják a sudo-t.
    template:
      src: sudo-group.j2
      dest: /etc/sudoers.d/sudo-group
      mode: 0440

Writing ./passwordless-sudo.yml


In [25]:
!ansible -i ./hosts --become -m apt -a"name=mc state=present" amazon1

amazon1 | SUCCESS => {
    "cache_update_time": 1521361316,
    "cache_updated": false,
    "changed": false
}


In [26]:
!ansible -i ./hosts --become -m ping -vvv amazon1

ansible 2.4.3.0
  config file = /home/sis/ubuntu-iso/ansible.cfg
  configured module search path = ['/home/sis/.ansible/plugins/modules', '/usr/share/ansible/plugins/modules']
  ansible python module location = /usr/local/lib/python3.5/dist-packages/ansible
  executable location = /usr/local/bin/ansible
  python version = 3.5.2 (default, Nov 23 2017, 16:37:01) [GCC 5.4.0 20160609]
Using /home/sis/ubuntu-iso/ansible.cfg as config file
Parsed /home/sis/ubuntu-iso/hosts inventory source with ini plugin
META: ran handlers
Using module file /usr/local/lib/python3.5/dist-packages/ansible/modules/system/ping.py
<52.59.217.167> ESTABLISH SSH CONNECTION FOR USER: ubuntu
<52.59.217.167> SSH: EXEC ssh -C -o ControlMaster=auto -o ControlPersist=60s -o StrictHostKeyChecking=no -o Port=22 -o 'IdentityFile="/home/sis/.ssh/amazon1.pem"' -o KbdInteractiveAuthentication=no -o PreferredAuthentications=gssapi-with-mic,gssapi-keyex,hostbased,publickey -o PasswordAuthentication=no -o User=ubuntu -o ConnectT

In [27]:
%%file ./index.html
<html>
valami
</html>

Writing ./index.html


In [28]:
%%file ./nginx.yml
---
- name: Install nginx
  hosts: amazon1
  become: true

  tasks:
  
  - name: Install nginx
    apt:
      name: nginx
      state: present

  - name: Insert Index Page
    template:
      src: index.html
      dest: /usr/share/nginx/html/index.html

  - name: Start NGiNX
    service:
      name: nginx
      state: started


Writing ./nginx.yml


In [29]:
!ansible-playbook -i ./hosts ./nginx.yml -l amazon


PLAY [Install nginx] ***********************************************************

TASK [Gathering Facts] *********************************************************
ok: [amazon1]

TASK [Install nginx] ***********************************************************
ok: [amazon1]

TASK [Insert Index Page] *******************************************************
ok: [amazon1]

TASK [Start NGiNX] *************************************************************
ok: [amazon1]

PLAY RECAP *********************************************************************
amazon1                    : ok=4    changed=0    unreachable=0    failed=0   

